# Importaciones

In [29]:
from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold
import numpy as np
from sklearn.model_selection import train_test_split
import time

In [30]:
def error_measures(Yestimado, Yteorico):
    
    CM = confusion_matrix(Yteorico, Yestimado)

    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    
    sens = TP/(TP+FN)
    esp = TN/(TN+FP)
    
    return sens, esp

In [31]:
def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

In [32]:
df = pd.read_excel('Opiniones_Hoteles_Medellin.xlsx')
df.columns=['texto','sentimiento']

In [33]:
X=df.iloc[:,0]
y=df.iloc[:,1]

In [34]:
vector=CountVectorizer(ngram_range=(1, 2))

In [35]:
vector.fit(X)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [36]:
bagOfWords=vector.transform(X)

# Regresión logística

In [37]:
from sklearn.linear_model import LogisticRegression
import scipy as sc

In [38]:
def RegresionLogistica(Cvalue):    
    kf = KFold(n_splits=10)
    lr = LogisticRegression(C = Cvalue, max_iter=200)
    acc = []
    sens = []
    esp = []
    Errores = np.ones(10)
    tiempo_i = time.time()
    
    j=0
    for train_index, test_index in kf.split(bagOfWords):
        Xtrain, Xtest = bagOfWords[train_index], bagOfWords[test_index]
        Ytrain, Ytest = y[train_index], y[test_index]  

        lr.fit(Xtrain,Ytrain)
        Yest = lr.predict(Xtest)
        s, e = error_measures(Yest, Ytest)
        sens.append(s); esp.append(e)
        acc.append(lr.score(Xtest,Ytest))   
        
        Errores[j] = classification_error(Yest, Ytest)
        j+=1

    #print("Accuracy: ", np.mean(acc), "+/-", np.std(acc))
    #print("Sensitivity: ", np.mean(sens), "+/-", np.std(sens))
    #print("Especificity: ", np.mean(esp), "+/-", np.std(esp))
    return np.mean(acc), np.std(acc), np.mean(sens), np.std(sens), np.mean(esp), np.std(esp), np.mean(Errores), time.time()-tiempo_i

In [39]:
import pandas as pd
import qgrid
randn = np.random.randn

tasas = pd.Series(['1.0', '0.1','0.001'])

df_types = pd.DataFrame({
    'Tasa de aprendizaje' : tasas})

df_types["Accuracy"] = ""
df_types["Int_Accuracy"] = ""
df_types["Sensibility"] = ""
df_types["Int_Sensibility"] = ""
df_types["Especificity"] = ""
df_types["Int_Especificity"] = ""
df_types["Error validación"] = ""
df_types["Tiempo ejecución"] = ""
df_types.set_index(['Tasa de aprendizaje'], inplace=True)
i=0

for eta in [1.0, 0.1, 0.001]:
    Acc, IntAcc, Sen, IntSen, Esp, IntEsp, error, tiempo=RegresionLogistica(eta)       
    df_types["Accuracy"][i] = Acc
    df_types["Int_Accuracy"][i] = IntAcc
    df_types["Sensibility"][i] = Sen
    df_types["Int_Sensibility"][i] = IntSen
    df_types["Especificity"][i] = Esp
    df_types["Int_Especificity"][i] = IntEsp
    df_types["Error validación"][i] = error
    df_types["Tiempo ejecución"][i] = tiempo
    i=i+1
        

# df_types["Error_Entrenamiento"][2] = "0.0"
# df_types["Error_Prueba"][2] = "0.5"
#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

C:\Users\sanqu\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\sanqu\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in longlong_scalars
  # Remove the CWD from sys.path while we load stuff.
C:\Users\sanqu\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in longlong_scalars
  # This is added back by InteractiveShellApp.init_path()


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [40]:
qgrid_widget.get_changed_df()

,Accuracy,Int_Accuracy,Sensibility,Int_Sensibility,Especificity,Int_Especificity,Error validación,Tiempo ejecución
Tasa de aprendizaje,,,,,,,,
1.0,0.848605,0.0593352,NaN,NaN,NaN,NaN,0.151395,0.20188
0.1,0.840799,0.0663257,NaN,NaN,NaN,NaN,0.159201,0.157913
0.001,0.641516,0.191019,NaN,NaN,NaN,NaN,0.358484,0.10094


# Árbol de decisión


In [41]:
from sklearn.tree import DecisionTreeClassifier as DT

In [42]:
def arbol_decision(prof):
    if(prof == 0):
        DT_model = DT(max_depth=None) 
    else:
        DT_model = DT(max_depth = prof)
    
    acc = []
    sens = []
    esp = []
    Errores = np.ones(100)
    tiempo_i = time.time()
    
    j=0
    for i in range(100):

        Xtrain,Xtest,Ytrain,Ytest = train_test_split(bagOfWords,y)   #Realiza una única partición de la base de datos

        DT_model.fit(Xtrain,Ytrain)
        Yest = DT_model.predict(Xtest)
        s, e = error_measures(Yest,Ytest)
        sens.append(s); esp.append(e)
        acc.append(DT_model.score(Xtest,Ytest))
        
        Errores[j] = classification_error(Yest, Ytest)
        j+=1

    #print("Accuracy: ", np.mean(acc), "+/-", np.std(acc))
    #print("Sensitivity: ", np.mean(sens), "+/-", np.std(sens))
    #print("Especificity: ", np.mean(esp), "+/-", np.std(esp))
    return np.mean(acc), np.std(acc), np.mean(sens), np.std(sens), np.mean(esp), np.std(esp),np.mean(Errores), time.time()-tiempo_i

In [43]:
randn = np.random.randn

df_types = pd.DataFrame({
    'Maxima profundidad' : pd.Series([0, 5,10,20,30,50,60,70])})

df_types["Accuracy"] = ""
df_types["Int_Accuracy"] = ""
df_types["Sensibility"] = ""
df_types["Int_Sensibility"] = ""
df_types["Especificity"] = ""
df_types["Int_Especificity"] = ""
df_types["Error validación"] = ""
df_types["Tiempo ejecución"] = ""
df_types.set_index(['Maxima profundidad'], inplace=True)

for i in df_types.index:
    Acc, IntAcc, Sen, IntSen, Esp, IntEsp, error, tiempo=arbol_decision(i)       
    df_types["Accuracy"][i] = Acc
    df_types["Int_Accuracy"][i] = IntAcc
    df_types["Sensibility"][i] = Sen
    df_types["Int_Sensibility"][i] = IntSen
    df_types["Especificity"][i] = Esp
    df_types["Int_Especificity"][i] = IntEsp
    df_types["Error validación"][i] = error
    df_types["Tiempo ejecución"][i] = tiempo
        
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [44]:
qgrid_widget.get_changed_df()

,Accuracy,Int_Accuracy,Sensibility,Int_Sensibility,Especificity,Int_Especificity,Error validación,Tiempo ejecución
Maxima profundidad,,,,,,,,
0,0.802171,0.0348489,0.779088,0.0503619,0.825685,0.0602123,0.197829,4.72729
5,0.800465,0.0365185,0.7507,0.0625269,0.852246,0.0840893,0.199535,2.72745
10,0.80155,0.0313546,0.759509,0.0543796,0.844628,0.055609,0.19845,3.75385
20,0.805736,0.0309655,0.783405,0.0533935,0.829808,0.0579112,0.194264,4.38047
30,0.803643,0.0359117,0.794622,0.0524795,0.8141,0.0606948,0.196357,4.63034
50,0.806977,0.0347456,0.786265,0.0477601,0.828607,0.0599895,0.193023,4.58037
60,0.800233,0.0319405,0.775901,0.0507578,0.826427,0.0587196,0.199767,4.5554
70,0.805891,0.0303995,0.789353,0.0513622,0.822103,0.0524168,0.194109,4.94414


# Random forest


In [45]:
from sklearn.ensemble import RandomForestClassifier as RF

In [46]:
def random_forest(est, carac):
    
    RF_model = RF(n_estimators=est, max_features =carac)
    acc = []
    sens = []
    esp = []
    Errores = np.ones(100)
    tiempo_i = time.time()
    
    j=0

    for i in range(100):
        Xtrain,Xtest,Ytrain,Ytest = train_test_split(bagOfWords,y)   #Realiza una única partición de la base de datos

        RF_model.fit(Xtrain,Ytrain)
        Yest = RF_model.predict(Xtest)
        s, e = error_measures(Yest,Ytest)
        sens.append(s); esp.append(e)
        acc.append(RF_model.score(Xtest,Ytest))
        Errores[j] = classification_error(Yest, Ytest)
        j+=1
        
    return np.mean(acc), np.std(acc), np.mean(sens), np.std(sens), np.mean(esp), np.std(esp),np.mean(Errores), time.time()-tiempo_i

In [47]:
randn = np.random.randn

df_types = pd.DataFrame({
    'Numero de arboles' : pd.Series([5,5,5,5,5,5,10,10,10,10,10,10,20,20,20,20,20,20,50,50,50,50,50,50,100,100,100,100,100,100]), 
    'Variables analizadas por nodo' : pd.Series([50,100,150,200,250,300,50,100,150,200,250,300,50,100,150,200,250,300,50,100,150,200,250,300,50,100,150,200,250,300])})

df_types["Accuracy"] = ""
df_types["Int_Accuracy"] = ""
df_types["Sensibility"] = ""
df_types["Int_Sensibility"] = ""
df_types["Especificity"] = ""
df_types["Int_Especificity"] = ""
df_types["Error validación"] = ""
df_types["Tiempo ejecución"] = ""
df_types.set_index(['Numero de arboles', 'Variables analizadas por nodo'], inplace=True)

for i in df_types.index:
    Acc, IntAcc, Sen, IntSen, Esp, IntEsp, error, tiempo=random_forest(i[0], i[1])       
    df_types["Accuracy"][i] = Acc
    df_types["Int_Accuracy"][i] = IntAcc
    df_types["Sensibility"][i] = Sen
    df_types["Int_Sensibility"][i] = IntSen
    df_types["Especificity"][i] = Esp
    df_types["Int_Especificity"][i] = IntEsp
    df_types["Error validación"][i] = error
    df_types["Tiempo ejecución"][i] = tiempo
        
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [48]:
qgrid_widget.get_changed_df()

Accuracy Int_Accuracy  \
Numero de arboles Variables analizadas por nodo                          
5                 50                             0.786357    0.0425343   
                  100                            0.791395    0.0382147   
                  150                            0.810078     0.035583   
                  200                            0.804729     0.036733   
                  250                            0.809225    0.0340008   
                  300                            0.809845    0.0329153   
10                50                             0.820388    0.0338087   
                  100                            0.831085    0.0309941   
                  150                            0.832171     0.036432   
                  200                            0.836202    0.0377807   
                  250                            0.832946     0.036399   
                  300                             0.83814    0.0262945   
20                50                              0.85845    0.0296732   
                  100                            0.852093    0.0311948   
                  150                             0.85124    0.0309825   
                  200                            0.858527    0.0308985   
                  250                            0.852791    0.0295489   
                  300                            0.849922    0.0287245   
50                50                             0.867752    0.0290221   
                  100                            0.866434    0.0254248   
                  150                            0.864961    0.0290709   
                  200                            0.862791    0.0315743   
                  250                            0.866667    0.0284191   
                  300                            0.861473    0.0310204   
100               50                             0.879302    0.0284667   
                  100                            0.866279    0.0322685   
                  150                            0.867984    0.0305677   
                  200                            0.869767    0.0267189   
                  250                             0.85969    0.0259314   
                  300                            0.867752    0.0300395   

                                                Sensibility Int_Sensibility  \
Numero de arboles Variables analizadas por nodo                               
5                 50                               0.822353       0.0704729   
                  100                              0.810732       0.0643881   
                  150                              0.824075       0.0634696   
                  200                              0.818463       0.0653143   
                  250                              0.829444       0.0655292   
                  300                              0.806363       0.0657303   
10                50                               0.800816       0.0671238   
                  100                              0.802674       0.0521096   
                  150                              0.803702       0.0709142   
                  200                              0.799591       0.0723005   
                  250                              0.802205       0.0645843   
                  300                              0.809419       0.0636516   
20                50                               0.871437       0.0502087   
                  100                               0.84695       0.0539085   
                  150                              0.840588       0.0564395   
                  200                              0.847917       0.0543829   
                  250                               0.83541       0.0603981   
                  300                              0.839185       0.0480671   
50                50                               0.888809       0.0470448   
                  100

# Support vector machine

In [49]:
from sklearn import svm

In [50]:
def SVM (ker, c, gam): 
    
    if gam == 0:
        gam = 'auto'
    
    svm_model =svm.SVC(kernel=ker, C=c, gamma=gam)
    acc = []
    sens = []
    esp = []
    Errores = np.ones(100)
    tiempo_i = time.time()
    
    j=0

    for i in range(100):

        Xtrain,Xtest,Ytrain,Ytest = train_test_split(bagOfWords,y)   #Realiza una única partición de la base de datos

        svm_model.fit(Xtrain,Ytrain)
        Yest = svm_model.predict(Xtest)
        s, e = error_measures(Yest,Ytest)
        sens.append(s); esp.append(e)
        acc.append(svm_model.score(Xtest,Ytest))
        Errores[j] = classification_error(Yest, Ytest)
        j+=1

    return np.mean(acc), np.std(acc), np.mean(sens), np.std(sens), np.mean(esp), np.std(esp),np.mean(Errores), time.time()-tiempo_i

In [51]:
randn = np.random.randn

df_types = pd.DataFrame({
    'Kernel' : pd.Series(['linear','linear','linear','linear','linear','linear','linear','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf', 'rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,200,0.001,0.001,0.001,0.001,0.01,0.01,0.01,0.01,0.1,0.1,0.1,0.1,1,1,1,1,10,10,10,10,100,100,100,100,200,200,200,200]),
    'gamma' : pd.Series([0,0,0,0,0,0,0,0.001,0.01,0.1,1,0.001,0.01,0.1,1,0.001,0.01,0.1,1,0.001,0.01,0.1,1,0.001,0.01,0.1,1,0.001,0.01,0.1,1,0.001,0.01,0.1,1])})

df_types["Accuracy"] = ""
df_types["Int_Accuracy"] = ""
df_types["Sensibility"] = ""
df_types["Int_Sensibility"] = ""
df_types["Especificity"] = ""
df_types["Int_Especificity"] = ""
df_types["Error validación"] = ""
df_types["Tiempo ejecución"] = ""
df_types.set_index(['Kernel','C','gamma'], inplace=True)

for i in df_types.index:
    Acc, IntAcc, Sen, IntSen, Esp, IntEsp, error, tiempo=SVM(i[0], i[1], i[2])       
    df_types["Accuracy"][i] = Acc
    df_types["Int_Accuracy"][i] = IntAcc
    df_types["Sensibility"][i] = Sen
    df_types["Int_Sensibility"][i] = IntSen
    df_types["Especificity"][i] = Esp
    df_types["Int_Especificity"][i] = IntEsp
    df_types["Error validación"][i] = error
    df_types["Tiempo ejecución"][i] = tiempo
        
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [52]:
qgrid_widget.get_changed_df()

Accuracy Int_Accuracy Sensibility Int_Sensibility  \
Kernel C       gamma                                                      
linear 0.001   0.000  0.615426    0.0962109    0.902022        0.224224   
       0.010   0.000  0.803023    0.0400423    0.933274       0.0324979   
       0.100   0.000  0.871163    0.0287508    0.904292       0.0382995   
       1.000   0.000  0.881395    0.0266175    0.894198        0.039705   
       10.000  0.000  0.875814    0.0260811    0.889475       0.0413947   
       100.000 0.000  0.877597    0.0269494    0.891442        0.040678   
       200.000 0.000   0.88031    0.0254358    0.885681       0.0399255   
rbf    0.001   0.001  0.472791    0.0242053        0.62        0.485386   
               0.010  0.467519    0.0269137        0.62        0.485386   
               0.100   0.47031    0.0244415        0.54        0.498397   
               1.000  0.471938    0.0228723        0.55        0.497494   
       0.010   0.001  0.468372    0.0266779        0.59        0.491833   
               0.010  0.470465    0.0235816        0.58        0.493559   
               0.100  0.466589    0.0245308        0.54        0.498397   
               1.000  0.469535    0.0227399        0.57        0.495076   
       0.100   0.001  0.469845    0.0209129        0.56        0.496387   
               0.010  0.654729    0.0587556    0.808776        0.150998   
               0.100   0.47093    0.0258821    0.521481        0.498081   
               1.000   0.47031    0.0211744        0.51          0.4999   
       1.000   0.001      0.69    0.0608169    0.945392       0.0313425   
               0.010  0.822481    0.0312875    0.895011       0.0386019   
               0.100  0.706822    0.0476146    0.555919       0.0769925   
               1.000   0.48155    0.0297674    0.686545        0.457107   
       10.000  0.001  0.835271    0.0319503    0.921024       0.0286763   
               0.010  0.873643    0.0261621    0.866161       0.0447594   
               0.100  0.737442    0.0368793    0.562955       0.0538452   
               1.000  0.479225    0.0266148    0.509477        0.489755   
       100.000 0.001  0.885581    0.0279311     0.89312       0.0375383   
               0.010  0.877132    0.0237005     0.86643       0.0429889   
               0.100  0.737442    0.0392168    0.569822       0.0632382   
               1.000   0.47907    0.0256635    0.608418        0.479725   
       200.000 0.001  0.877829    0.0291728    0.881611       0.0468658   
               0.010  0.879845    0.0262767    0.873358       0.0388246   
               0.100  0.727442    0.0387863    0.550693        0.058042   
               1.000  0.477829    0.0293044    0.548152        0.488509   

                     Especificity Int_Especificity Error validación  \
Kernel C       gamma                                                  
linear 0.001   0.000     0.354837         0.282498         0.384574   
       0.010   0.000     0.673946        0.0662476         0.196977   
       0.100   0.000     0.839947        0.0503959         0.128837   
       1.000   0.000     0.870449          0.04176         0.118605   
       10.000  0.000     0.863173        0.0444681         0.124186   
       100.000 0.000     0.864878        0.0403132         0.122403   
       200.000 0.000     0.875667        0.0417564          0.11969   
rbf    0.001   0.001         0.38         0.485386         0.527209   
               0.010         0.38         0.485386         0.532481   
               0.100         0.46         0.498397          0.52969   
               1.000         0.45         0.497494         0.528062   
       0.010   0.001         0.41         0.491833         0.531628   
               0.010         0.42         0.493559         0.529535   
               0.100         0.46         0.498397         0.533411   
               1.000         0.43         0.495076         0.530465   
       0.100   0.001         0.44      

# Red neuronal

In [53]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold

In [54]:
def red_neuronal(nCO, nPC):
    if(nCO==1):
        mlp=MLPClassifier(activation='tanh',max_iter = 500,hidden_layer_sizes=(nPC))
    else:
        mlp=MLPClassifier(activation='tanh',max_iter = 500,hidden_layer_sizes=(nPC,nPC))
        
    acc = []
    sens = []
    esp = []
    Errores = np.ones(100)
    tiempo_i = time.time()
    
    j=0

    for i in range(100):

        Xtrain,Xtest,Ytrain,Ytest = train_test_split(bagOfWords,y)   #Realiza una única partición de la base de datos

        mlp.fit(Xtrain,Ytrain)
        Yest = mlp.predict(Xtest)
        s, e = error_measures(Yest,Ytest)
        sens.append(s); esp.append(e)
        acc.append(mlp.score(Xtest,Ytest))
        Errores[j] = classification_error(Yest, Ytest)
        j+=1

    return np.mean(acc), np.std(acc), np.mean(sens), np.std(sens), np.mean(esp), np.std(esp),np.mean(Errores), time.time()-tiempo_i

In [55]:
randn = np.random.randn

df_types = pd.DataFrame({
    'N. de capas ocultas' : pd.Series([1,1,1,1,1,2,2,2,2,2]),
    'Neuronas por capa' : pd.Series([20,24,28,32,36,20,24,28,32,36])})

df_types["Accuracy"] = ""
df_types["Int_Accuracy"] = ""
df_types["Sensibility"] = ""
df_types["Int_Sensibility"] = ""
df_types["Especificity"] = ""
df_types["Int_Especificity"] = ""
df_types["Error validación"] = ""
df_types["Tiempo ejecución"] = ""
df_types.set_index(['N. de capas ocultas','Neuronas por capa'], inplace=True)

for i in df_types.index:
    Acc, IntAcc, Sen, IntSen, Esp, IntEsp, error, tiempo=red_neuronal(i[0], i[1])       
    print(i[0], i[1])
    df_types["Accuracy"][i] = Acc
    df_types["Int_Accuracy"][i] = IntAcc
    df_types["Sensibility"][i] = Sen
    df_types["Int_Sensibility"][i] = IntSen
    df_types["Especificity"][i] = Esp
    df_types["Int_Especificity"][i] = IntEsp
    df_types["Error validación"][i] = error
    df_types["Tiempo ejecución"][i] = tiempo
        
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

1 20
1 24
1 28
1 32
1 36
2 20
2 24
2 28
2 32
2 36


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [56]:
qgrid_widget.get_changed_df()

Accuracy Int_Accuracy Sensibility  \
N. de capas ocultas Neuronas por capa                                      
1                   20                 0.892326    0.0235357    0.924016   
                    24                 0.894574    0.0246604    0.923712   
                    28                 0.895349     0.027176    0.927015   
                    32                 0.891473    0.0264476    0.924838   
                    36                 0.894109    0.0243717    0.926542   
2                   20                 0.902403    0.0268332    0.925485   
                    24                 0.903178    0.0235254    0.925731   
                    28                 0.903023    0.0240059    0.925914   
                    32                 0.902171    0.0249458    0.915581   
                    36                 0.908915    0.0256488    0.919194   

                                      Int_Sensibility Especificity  \
N. de capas ocultas Neuronas por capa                                
1                   20                      0.0319172     0.861255   
                    24                      0.0340945     0.867072   
                    28                       0.032485     0.864443   
                    32                      0.0320284     0.858225   
                    36                      0.0329527     0.863052   
2                   20                      0.0371121     0.879929   
                    24                       0.032058     0.880733   
                    28                      0.0344023     0.880924   
                    32                      0.0350105     0.888567   
                    36                      0.0327905     0.899328   

                                      Int_Especificity Error validación  \
N. de capas ocultas Neuronas por capa                                     
1                   20                       0.0442534         0.107674   
                    24                       0.0427562         0.105426   
                    28                        0.049903         0.104651   
                    32                       0.0477324         0.108527   
                    36                       0.0447215         0.105891   
2                   20                       0.0476317        0.0975969   
                    24                       0.0463686        0.0968217   
                    28                       0.0410637        0.0969767   
                    32                       0.0438096        0.0978295   
                    36                       0.0411841        0.0910853   

                                      Tiempo ejecución  
N. de capas ocultas Neuronas por capa                   
1                   20                         760.853  
                    24                         772.201  
                    28                         839.394  
                    32                         971.446  
                    36                          1030.9  
2                   20                         501.831  
                    24                         538.825  
                    28                          573.21  
                    32                         611.918  
                    36                         598.158